In [1]:
import os
import tempfile
import numpy as np
import tensorflow as tf
print(tf.__version__)

2.8.2


# 1. pb forzen model

download the model

In [4]:
import wget

In [6]:
MODEL_NAME = 'mobilenetv2_dm05_coco_voc_trainval'  # @param ['mobilenetv2_dm05_coco_voc_trainval', 'mobilenetv2_coco_voctrainval', 'xception_coco_voctrainval']

DOWNLOAD_URL_PREFIX = 'http://download.tensorflow.org/models/'
MODEL_URLS = {
    'mobilenetv2_dm05_coco_voc_trainval':
        'deeplabv3_mnv2_dm05_pascal_trainval_2018_10_01.tar.gz',
    'mobilenetv2_coco_voctrainval':
        'deeplabv3_mnv2_pascal_train_aug_2018_01_29.tar.gz',
    'xception_coco_voctrainval':
        'deeplabv3_pascal_trainval_2018_01_04.tar.gz',
}

MODEL_TAR = MODEL_URLS[MODEL_NAME]
MODEL_URL = DOWNLOAD_URL_PREFIX + MODEL_TAR
print(MODEL_TAR, MODEL_URL)

deeplabv3_mnv2_dm05_pascal_trainval_2018_10_01.tar.gz http://download.tensorflow.org/models/deeplabv3_mnv2_dm05_pascal_trainval_2018_10_01.tar.gz


In [7]:
wget.download(MODEL_URL, MODEL_TAR)

 22% [................                                                          ] 1785856 / 7976055

100% [..........................................................................] 7976055 / 7976055

'deeplabv3_mnv2_dm05_pascal_trainval_2018_10_01.tar.gz'

unzip (manually with 7zip) and get the file location

In [12]:
MODEL_FILE = "deeplabv3_mnv2_dm05_pascal_trainval/frozen_model/frozen_inference_graph.pb"
print('Frozen graph file path:', MODEL_FILE)

Frozen graph file path: deeplabv3_mnv2_dm05_pascal_trainval/saved_model/frozen_inference_graph.pb


### load the graph and check the output node

In [13]:
with tf.compat.v1.Session() as sess:
    print("load graph")
    with tf.io.gfile.GFile(MODEL_FILE,'rb') as f:
        graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())
    sess.graph.as_default()
    tf.import_graph_def(graph_def, name='')
    graph_nodes=[n for n in graph_def.node]
    names = []
    for t in graph_nodes:
        names.append(t.name)
    print(names)

load graph
['ImageTensor', 'Shape', 'strided_slice/stack', 'strided_slice/stack_1', 'strided_slice/stack_2', 'strided_slice', 'Squeeze', 'Cast', 'Shape_1', 'strided_slice_1/stack', 'strided_slice_1/stack_1', 'strided_slice_1/stack_2', 'strided_slice_1', 'strided_slice_2/stack', 'strided_slice_2/stack_1', 'strided_slice_2/stack_2', 'strided_slice_2', 'sub/x', 'sub', 'Maximum/y', 'Maximum', 'add', 'sub_1/x', 'sub_1', 'Maximum_1/y', 'Maximum_1', 'add_1', 'Reshape/tensor', 'Reshape/shape', 'Reshape', 'Rank', 'Equal/y', 'Equal', 'Assert/Assert/data_0', 'Assert/Assert/data_1', 'Assert/Assert', 'sub_2', 'Shape_2', 'strided_slice_3/stack', 'strided_slice_3/stack_1', 'strided_slice_3/stack_2', 'strided_slice_3', 'strided_slice_4/stack', 'strided_slice_4/stack_1', 'strided_slice_4/stack_2', 'strided_slice_4', 'GreaterEqual', 'Assert_1/Assert/data_0', 'Assert_1/Assert', 'GreaterEqual_1', 'Assert_2/Assert/data_0', 'Assert_2/Assert', 'sub_3/y', 'sub_3', 'sub_4', 'sub_5/y', 'sub_5', 'sub_6', 'Greate

the output tensor is resizebilinear_2

<b>no need to run this part</b>

see https://colab.research.google.com/github/sayakpaul/Adventures-in-TensorFlow-Lite/blob/master/DeepLabV3/DeepLab_TFLite_COCO.ipynb

In [9]:
# Load the TensorFlow model
# The preprocessing and the post-processing steps should not be included in the TF Lite model graph 
# because some operations (ArgMax) might not support the delegates. 
# Insepct the graph using Netron https://lutzroeder.github.io/netron/
converter = tf.compat.v1.lite.TFLiteConverter.from_frozen_graph(
    graph_def_file = MODEL_FILE, 
    input_arrays = ['sub_7'],
    output_arrays = ['ResizeBilinear_2']
)

# Optional: Perform the simplest optimization known as post-training dynamic range quantization.
# https://www.tensorflow.org/lite/performance/post_training_quantization#dynamic_range_quantization
# You can refer to the same document for other types of optimizations.
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Convert to TFLite Model
tflite_model = converter.convert()

_, dynamic_tflite_path = tempfile.mkstemp('.tflite')
tflite_model_size = open(dynamic_tflite_path, 'wb').write(tflite_model)
tf_model_size = os.path.getsize(MODEL_FILE)
print('TensorFlow Model is  {} bytes'.format(tf_model_size))
print('TFLite Model is      {} bytes'.format(tflite_model_size))
print('Post training dynamic range quantization saves {} bytes'.format(tf_model_size-tflite_model_size))

TensorFlow Model is  3042785 bytes
TFLite Model is      936592 bytes
Post training dynamic range quantization saves 2106193 bytes


run the following command in ternminal to convert the model

<b>do not run this in jupyter</b>

see https://www.tensorflow.org/js/tutorials/conversion/import_saved_model

In [ ]:
tensorflowjs_converter --input_format=tf_frozen_model --output_node_names='ResizeBilinear_2' deeplabv3_mnv2_dm05_pascal_trainval/frozen_inference_graph.pb deeplabv3_mnv2_dm05_pascal_trainval/web

# 2. saved model

In [2]:
MODEL_FILE = "ssd_mobilenet_v1_coco2017/saved_model"

In [3]:
model = tf.keras.models.load_model(MODEL_FILE)

In [10]:
dir(model)

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_trackable_child',
 '_add_variable_with_custom_getter',
 '_checkpoint_dependencies',
 '_deferred_dependencies',
 '_delete_tracking',
 '_deserialization_dependencies',
 '_deserialize_from_proto',
 '_gather_saveables_for_checkpoint',
 '_get_legacy_saved_model_children',
 '_handle_deferred_dependencies',
 '_list_extra_dependencies_for_serialization',
 '_list_functions_for_serialization',
 '_lookup_dependency',
 '_map_resources',
 '_maybe_initialize_trackable',
 '_model',
 '_name_based_attribute_restore',
 '_name_based_restores',
 '_no_dependency',
 '_object_identifier',
 '_preload_simple_restoration',
 '_re

# 3. tf hub model

In [30]:
import tensorflow_hub as hub
import numpy as np
import tensorflowjs as tfjs

In [17]:
detector = hub.KerasLayer("https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1",
                          trainable=False,
                          signature="default",
                          signature_outputs_as_dict=True)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [22]:
detector.summary()

AttributeError: 'KerasLayer' object has no attribute 'summary'

In [23]:
input_layer = tf.keras.Input(shape=[256,256,3])

In [24]:
output_layer=detector(input_layer)

In [25]:
output_layer

{'detection_class_entities': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'keras_layer_4')>,
 'detection_boxes': <KerasTensor: shape=(None, 4) dtype=float32 (created by layer 'keras_layer_4')>,
 'detection_class_names': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'keras_layer_4')>,
 'detection_class_labels': <KerasTensor: shape=(None, 1) dtype=int64 (created by layer 'keras_layer_4')>,
 'detection_scores': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'keras_layer_4')>}

In [27]:
model = tf.keras.models.Model(input_layer,output_layer)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 keras_layer_4 (KerasLayer)  {'detection_class_entiti  0         
                             es': (None, 1),                     
                              'detection_boxes': (Non            
                             e, 4),                              
                              'detection_class_names'            
                             : (None, 1),                        
                              'detection_class_labels            
                             ': (None, 1),                       
                              'detection_scores': (No            
                             ne, 1)}                             
                                                             

In [31]:
tfjs.converters.save_keras_model(model, 'test')

In [33]:
detector = hub.load("https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1").signatures['default']

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [37]:
dir(detector.graph.outputs)

['_ControlDependenciesController',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_control_dependencies',
 '_add_device_to_stack',
 '_add_function',
 '_add_new_tf_operations',
 '_add_op',
 '_add_scope_exit_callback',
 '_apply_device_functions',
 '_as_graph_def',
 '_as_graph_element_locked',
 '_attr_scope',
 '_attr_scope_map',
 '_bcast_grad_args_cache',
 '_building_function',
 '_c_graph',
 '_capture_by_value',
 '_capture_helper',
 '_captures',
 '_check_not_finalized',
 '_collections',
 '_colocate_with_for_gradient',
 '_colocation_stack',
 '_container',
 '_control_dependencies_for_inputs',
 '_control_dependencies_stack',
 '_control_flow_context',
 '_copy_functions_

In [38]:
detector.graph.outputs

[<tf.Tensor 'hub_input/strided_slice:0' shape=(None, 4) dtype=float32>,
 <tf.Tensor 'hub_input/index_to_string_Lookup:0' shape=(None, 1) dtype=string>,
 <tf.Tensor 'hub_input/strided_slice_2:0' shape=(None, 1) dtype=int64>,
 <tf.Tensor 'hub_input/index_to_string_1_Lookup:0' shape=(None, 1) dtype=string>,
 <tf.Tensor 'hub_input/strided_slice_1:0' shape=(None, 1) dtype=float32>]